In [2]:
import warnings
warnings.filterwarnings('ignore') # We can suppress the warnings

In [3]:
spark

In [4]:
import pandas as pd
import numpy as np
from PIL import Image
from io import BytesIO
import re
# import tensorflow as tf
# import cv2

I will use the functionality of Image Datasource of spark to collect and process images in bytes, and then start processing the images for the neural network

In [5]:
images_folders = ['/CA1/Images/BakedPotato/','/CA1/Images/Pizza/','/CA1/Images/Taco/']

In [6]:
images_rdd = spark.sparkContext.binaryFiles(','.join(images_folders))
# images_rdd = spark.sparkContext.binaryFiles('hdfs://172.24.144.178:9000/CA1/Images/Ireland/Ireland_001.jpeg')

In [7]:
images_rdd

org.apache.spark.api.java.JavaPairRDD@47722162

In [8]:
def extract_data(data):
    file_path, image_data = data
#     image = Image.open(BytesIO(image_data))
#     image_array = np.array(image)
    
    file_name = file_path.split('/')[-1]
    file_name_without_ext = file_name.split(".")[0]
    label,name = file_name_without_ext.split('_')[0], file_name_without_ext.split('_')[1]
    
    return name, label, image_data

In [9]:
imageDf = images_rdd.map(lambda x: extract_data(x)).toDF(["Name","Label","Data"])

In [10]:
pandasImagesDF = imageDf.toPandas()

In [11]:
IMG_SHAPE=225

In [12]:
def processImage(data, target_size=(IMG_SHAPE,IMG_SHAPE)):
    imgbytes = BytesIO(data)
    image = Image.open(imgbytes)
    resized_img = image.resize(target_size, Image.ANTIALIAS)
    with BytesIO() as output:
        resized_img.save(output, format="PNG")
        new_image = Image.open(output)

        array = np.asarray(new_image).reshape([target_size[0],target_size[1],3])
        return array

In [13]:
pandasImagesDF["Data"] = pandasImagesDF["Data"].apply(lambda x: processImage(x))

In [14]:
# pandasImagesDF

In [15]:
pandasImagesDF = pandasImagesDF.sort_values(by=["Name"])

In [16]:
pandasImagesDF.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3275 entries, 0 to 3274
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Name    3275 non-null   object
 1   Label   3275 non-null   object
 2   Data    3275 non-null   object
dtypes: object(3)
memory usage: 102.3+ KB


In [17]:
NCATEGORIES = len(pandasImagesDF["Label"].unique())

In [18]:
NCATEGORIES

3

Importing from Keras functionality necessary to implement CNN

In [19]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten, Input
from tensorflow.keras import utils
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

2024-03-22 23:53:39.173531: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-22 23:53:40.133782: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Now we are to split the data using train_test_split function to use Kfold and improve the perfomance of the model

In [20]:
X_train, X_test, y_train, y_test = train_test_split(pandasImagesDF["Data"],pandasImagesDF["Label"],test_size=0.3, random_state=42)

In [21]:
encoder = OneHotEncoder(sparse_output=False)
y_train = np.array(y_train).reshape(-1, 1)
y_train = encoder.fit_transform(y_train)

y_test = np.array(y_test).reshape(-1,1)
y_test = encoder.fit_transform(y_test)

In [22]:
X_train = np.stack(X_train)
X_test = np.stack(X_test)

In [23]:
X_train.shape

(2292, 225, 225, 3)

In [24]:
X_train = np.reshape(X_train,(X_train.shape[0],IMG_SHAPE,IMG_SHAPE,3)).astype(np.float32)
X_test = np.reshape(X_test,(X_test.shape[0],IMG_SHAPE,IMG_SHAPE,3)).astype(np.float32)

Because the pixels are from 0 to 255, We have to normalize the pixels

In [25]:
X_train/=255
X_test/=255

In [26]:
X_train.shape,X_test.shape

((2292, 225, 225, 3), (983, 225, 225, 3))

In [27]:
y_train.shape, y_test.shape

((2292, 3), (983, 3))

In [28]:
model = Sequential()
model.add(Input(shape=(IMG_SHAPE, IMG_SHAPE, 3)))  # 225x225 RGB images
model.add(Conv2D(32,kernel_size=(3,3),strides=(1,1),padding="valid", activation="relu"))
model.add(Conv2D(32,kernel_size=(3,3),activation="relu"))
model.add(MaxPool2D(3))
model.add(Conv2D(32, (3,3), activation="relu"))
model.add(Conv2D(32, (3,3), activation="relu"))
model.add(MaxPool2D(2))
model.add(Conv2D(32, (3,3), activation="relu"))
model.add(Conv2D(32, (3,3), activation="relu"))
model.add(MaxPool2D(2))
model.add(Conv2D(32, (3,3), activation="relu"))
model.add(Conv2D(32, (3,3), activation="relu"))
model.add(MaxPool2D(2))

model.add(Flatten())


model.add(Dense(128, activation="relu"))
model.add(Dense(NCATEGORIES,activation="softmax"))

model.compile(loss="category_crossentropy", metrics=["accuracy"], optimizer="adam")

2024-03-22 23:53:52.395416: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-22 23:53:52.612908: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-22 23:53:52.612959: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-22 23:53:52.617600: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-22 23:53:52.617715: I external/local_xla/xla/stream_executor

In [ ]:
model.summary()

In [1]:
model.fit(X_train, y_train, batch_size=32, epochs=20, verbose=1, validation_data=(X_test, y_test))

NameError: name 'model' is not defined

In [ ]:
pred = model.predict(X_test[:25])

In [ ]:
def comparePredictions(predictionsValues, expectedValues):
    correct = 0
    for x in range(len(predictionsValues)):
        if predictionsValues[x] == expectedValues[x]:
            correct+=1
    
    print(f'Correct predictions {correct}/{len(predictionsValues)}. Percent {correct/len(predictionsValues)}')

In [ ]:
comparePredictions(np.argmax(pred, axis=1),np.argmax(y_test[:25], axis=1))

### Trying Data Augmentation 

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(pandasImagesDF["Data"],pandasImagesDF["Label"],test_size=0.3, random_state=42)

In [ ]:
encoder = OneHotEncoder(sparse_output=False)
y_train = np.array(y_train).reshape(-1, 1)
y_train = encoder.fit_transform(y_train)

y_test = np.array(y_test).reshape(-1,1)
y_test = encoder.fit_transform(y_test)

X_train = np.stack(X_train)
X_train = np.reshape(X_train,(X_train.shape[0],IMG_SHAPE,IMG_SHAPE,3)).astype(np.float32)
X_train/=255

X_test = np.stack(X_test)
X_test = np.reshape(X_test,(X_test.shape[0],IMG_SHAPE,IMG_SHAPE,3)).astype(np.float32)
X_test/=255

In [ ]:
datagen = ImageDataGenerator(
    rotation_range=20,  # Random rotation between 0 and 20 degrees
    width_shift_range=0.1,  # Randomly shift the width by up to 10%
    height_shift_range=0.1,  # Randomly shift the height by up to 10%
    shear_range=0.2,  # Shear intensity (shear angle in counter-clockwise direction in radians)
    zoom_range=0.2,  # Randomly zoom in by up to 20%
    horizontal_flip=True,  # Randomly flip images horizontally
    fill_mode='nearest'  # Fill mode for points outside the input boundaries
)

In [ ]:
print("Shape of X_train:", X_train.shape)
print("Type of X_train:", type(X_train))
print("Shape of y_train:", y_train.shape)
print("Type of y_train:", type(y_train))

In [ ]:
#augmented_generator = datagen.flow(np.array(X_train), np.array(y_train), batch_size=32)

In [ ]:
#augmented_generator

In [ ]:
# # Get a batch of augmented data
# images, labels = next(augmented_generator)

# # Visualize the augmented images
# plt.figure(figsize=(10, 10))
# for i in range(9):
#     plt.subplot(3, 3, i + 1)
#     plt.imshow(images[i])
#     plt.title(f'Label: {labels[i]}')
#     plt.axis('off')
# plt.show()

In [ ]:
from tensorflow.keras.layers import RandomContrast, RandomBrightness, RandomRotation, RandomFlip

In [ ]:
dataAugmented = Sequential()
dataAugmented.add(RandomContrast(0.7))
dataAugmented.add(RandomBrightness(0.7))
dataAugmented.add(RandomRotation(0.4))
dataAugmented.add(RandomFlip())


In [ ]:
modelAugmented = Sequential()
modelAugmented.add(Input(shape=(IMG_SHAPE, IMG_SHAPE, 3)))  # 225x225 RGB images
modelAugmented.add(dataAugmented)
modelAugmented.add(Conv2D(32,kernel_size=(3,3),strides=(1,1),padding="valid", activation="relu"))
modelAugmented.add(Conv2D(32,kernel_size=(3,3),activation="relu"))
modelAugmented.add(MaxPool2D(3))
modelAugmented.add(Conv2D(32, (3,3), activation="relu"))
modelAugmented.add(Conv2D(32, (3,3), activation="relu"))
modelAugmented.add(MaxPool2D(2))
modelAugmented.add(Conv2D(32, (3,3), activation="relu"))
modelAugmented.add(Conv2D(32, (3,3), activation="relu"))
modelAugmented.add(MaxPool2D(2))
modelAugmented.add(Conv2D(32, (3,3), activation="relu"))
modelAugmented.add(Conv2D(32, (3,3), activation="relu"))
modelAugmented.add(MaxPool2D(2))

modelAugmented.add(Flatten())


modelAugmented.add(Dense(128, activation="relu"))
modelAugmented.add(Dense(NCATEGORIES,activation="sigmoid"))

modelAugmented.compile(loss="binary_crossentropy", metrics=["accuracy"], optimizer="adam")

In [ ]:
X_train.shape,X_test.shape, y_train.shape, y_test.shape

In [ ]:
modelAugmented.fit(X_train, y_train, batch_size=32, epochs=30, verbose=1, validation_data=(X_test, y_test))

In [ ]:
modelAugmented.summary()

In [ ]:
pred = modelAugmented.predict(X_test[:25])

In [ ]:
comparePredictions(np.argmax(pred, axis=1),np.argmax(y_test[:25], axis=1))